# Simplest use case

## Running a model from the config file

Setting up the `dolphin` ecosystem:

- Install `dolphin`, `lenstronomy`, and the required dependencies.
- Create an input/output directory for `dolphin`, we are using "../io_directory_example" in this example.
- Set up these directories inside the input/output directory. Look inside "../io_directory_example" for example. 
    - **data**: contains subdirectories for each lens system with a data and PSF files for each band.
    - **settings**: contains the 'config_{lens_name}.yml' files for each lens system,
    - **logs**: to write the log files from model runs,
    - **outputs**: to save the model outputs,
    - **hpc**: *optional*, contains scripts to submit batch jobs in MPI.
 

   
### Data format

The image data file needs to be in the hdf5 formate with the following datasets:

- `image_data`: reduced and background subtracted image cutout centered at the lens system,
- `background_rms`: background level,
- `exposure_time`: the map of exposure times for each pixel, so that `image_data * exposure_time` is Poisson noise distributed,
- `ra_at_xy_0`: RA of the (0, 0) pixel in the `image_data` cutout,
- `dec_at_xy_0`: Dec of the (0, 0) pixel in the `image_data` cutout,
- `transform_pix2angle`: a transform matrix to map the pixel numbers (x, y) to angles (RA, Dec).

The PSF data file needs to be in the hdf5 format with the following datasets:

- `kernel_point_source`: a pixelated PSF (not required to have the same dimension of `image_data`),
- `psf_error_map`: *optional*, uncertainty in the provided PSF, needs to have the same dimension of `kernel_point_source`. 

### Config file format

The 'config_{lens_name}.yml' file provides you options to set up the lens model. Here is the content of the config file that is used in the example model below.

### create a `Processor` instance and point to the IO directory

In [1]:
from dolphin.processor import Processor 
import dolphin
print(dolphin.processor.__file__)

processor = Processor('../io_directory_example/')


/Users/chinyi/Desktop/dolphin/dolphin/processor/__init__.py


### Run a model by calling the  `swim()` method

In [2]:
processor.swim(lens_name='SL2SJ0219-0829', model_id='example2', log=False,verbose=True)

lens_model_list: ['SPEP', 'SHEAR_GAMMA_PSI']

lens_light_model_list: ['SERSIC_ELLIPSE', 'SERSIC_ELLIPSE', 'SERSIC_ELLIPSE', 'SERSIC_ELLIPSE']
lens_light_indices: [[0, 1], [2, 3]]

source_light_model_list: ['SHAPELETS', 'SHAPELETS']
source_light_indices: [[0], [1]]

joint_lens_with_lens: []
joint_lens_light_with_lens: []
joint_lens_light_with_lens_light: [[0, 1, ['center_x', 'center_y']], [0, 2, ['center_x', 'center_y']], [0, 3, ['center_x', 'center_y']]]
joint_source_with_source: [[0, 1, ['center_x', 'center_y']]]
joint_source_with_point_source: []

nmax: [4, 4]
0 {'n_sersic': 4.0}
1 {'n_sersic': 3.5}
Computing the PSO ...
PSO: Exception while calling your likelihood function:
  params: [0.947523144439404, 0.06714452322192474, -0.09944905415984898, 0.08347693876132845, -0.10905610050349202, 0.0014783677862011473, -0.009431373872207539, 0.15000396969113364, -0.23806925463223483, 0.02630959616309958, 0.1493003927833561, 0.18166637683981895, -0.009017138917139317, 0.052059957150663555, 0.

Traceback (most recent call last):
  File "/Users/chinyi/Desktop/lenstronomy/lenstronomy/Sampling/Samplers/pso.py", line 446, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/Users/chinyi/Desktop/lenstronomy/lenstronomy/Sampling/likelihood.py", line 154, in logL
    return self.log_likelihood(kwargs_return, verbose=verbose)
  File "/Users/chinyi/Desktop/lenstronomy/lenstronomy/Sampling/likelihood.py", line 169, in log_likelihood
    logL_image = self.image_likelihood.logL(**kwargs_return)
  File "/Users/chinyi/Desktop/lenstronomy/lenstronomy/Sampling/Likelihoods/image_likelihood.py", line 51, in logL
    check_positive_flux=self._check_positive_flux)
  File "/Users/chinyi/Desktop/lenstronomy/lenstronomy/ImSim/MultiBand/multi_linear.py", line 100, in likelihood_data_given_model
    check_positive_flux=check_positive_flux)
  File "/Users/chinyi/Desktop/lenstronomy/lenstronomy/ImSim/MultiBand/single_band_multi_model.py", line 109, in likelihood_data_given_model
    lin

KeyboardInterrupt: 

## Let's check the output

In [ ]:
from dolphin.analysis.output import Output

output = Output('../io_directory_example/')

In [ ]:
fig = output.plot_model_overview(lens_name='SL2SJ0219-0829', model_id='example2',band_index=0)
fig2 = output.plot_model_overview(lens_name='SL2SJ0219-0829', model_id='example2',band_index=1)

We only ran a pre-sampling optimization here and did not perform a MCMC sampling. So, the above model is the optimized model that the MCMC sample can initiate from. The `kwargs_result` dictionary of the pre-sampling optimization step can be accessed through `Output.kwargs_result` after loading an output using the `Output.load_output()` method.

In [ ]:
output.load_output(lens_name='SL2SJ0219-0829', model_id='example2')

output.kwargs_result

When necessary, the settings of the model run---that was read out of the 'config_{lens_name}.yml' file---can be accessed through the `output.model_settings` variable.

In [ ]:
output.model_settings